In [100]:
print("SATELITE IMAGE CLASSIFICATION")

SATELITE IMAGE CLASSIFICATION


In [106]:
import torch 
import torchvision
from torchvision import models
import numpy as np
from PIL import Image
import torch.nn as nn
import os 
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torch.utils.data import random_split

In [102]:
path_lib = r"data"
classes = {}
sub_folder_name = [folder for folder in os.listdir(path_lib) if os.path.isdir(os.path.join(path_lib,folder))]

for i in range(len(sub_folder_name)):
    classes[sub_folder_name[i]] = i 

In [123]:
classes

{'cloudy': 0, 'desert': 1, 'green_area': 2, 'water': 3}

In [103]:
class SatelliteImageDataset(Dataset):
    def __init__(self,root_dir,transform=None,target_transform = None):
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.classes = classes
        self.image_path = []
        self.labels = []
    
        for label_name, label_value in self.classes.items():
            label_dir = os.path.join(self.root_dir,label_name)
            for file_name in os.listdir(label_dir):
                if file_name.endswith(('png','jpeg','jpg')):
                    self.image_path.append(os.path.join(label_dir,file_name))
                    self.labels.append(label_value)
        
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self, index):

        if torch.is_tensor(index):
            index = index.tolist()
        
        img_path = self.image_path[index]
        label = self.labels[index]

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label 

In [104]:
transform = transforms.Compose([
transforms.Grayscale(num_output_channels=3),
transforms.Resize((224,224)),
transforms.RandomHorizontalFlip(),
transforms.RandomRotation(degrees=10),
transforms.RandomResizedCrop(224),
transforms.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2,hue=0.2),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [105]:
dataset = SatelliteImageDataset(root_dir=r"data",transform=transform)

In [107]:
dataset_size = dataset.__len__()
train_data_size = int(0.8 * dataset_size)
val_data_size = dataset_size - train_data_size

train_dataset, test_dataset = random_split(dataset,[train_data_size,val_data_size])

In [108]:
image_train,label_train = train_dataset[110]
image_test, label_test = test_dataset[2]

In [109]:
print(image_test.shape,label_test)

torch.Size([3, 224, 224]) 1


In [110]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [111]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True)

In [112]:
model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = True

num_features = model.fc.in_features
model.fc = nn.Linear(num_features,4)


c:\Users\aarav\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aarav\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [113]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=7,gamma=0.1) 

In [114]:
num_epochs = 50
model = model.to(device)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        y_pred = model(images)
        loss = criterion(y_pred,labels)

        optimizer.zero_grad()

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    scheduler.step()

    print(f"::: Number of Epochs :::: {epoch} :::::::: Loss ::::: {running_loss/len(train_loader):.4f}")

::: Number of Epochs :::: 0 :::::::: Loss ::::: 0.5400
::: Number of Epochs :::: 1 :::::::: Loss ::::: 0.4126
::: Number of Epochs :::: 2 :::::::: Loss ::::: 0.3931
::: Number of Epochs :::: 3 :::::::: Loss ::::: 0.3523
::: Number of Epochs :::: 4 :::::::: Loss ::::: 0.2995
::: Number of Epochs :::: 5 :::::::: Loss ::::: 0.3077
::: Number of Epochs :::: 6 :::::::: Loss ::::: 0.3116
::: Number of Epochs :::: 7 :::::::: Loss ::::: 0.2383
::: Number of Epochs :::: 8 :::::::: Loss ::::: 0.1941
::: Number of Epochs :::: 9 :::::::: Loss ::::: 0.2078
::: Number of Epochs :::: 10 :::::::: Loss ::::: 0.1807
::: Number of Epochs :::: 11 :::::::: Loss ::::: 0.1746
::: Number of Epochs :::: 12 :::::::: Loss ::::: 0.1694
::: Number of Epochs :::: 13 :::::::: Loss ::::: 0.1587
::: Number of Epochs :::: 14 :::::::: Loss ::::: 0.1431
::: Number of Epochs :::: 15 :::::::: Loss ::::: 0.1311
::: Number of Epochs :::: 16 :::::::: Loss ::::: 0.1203
::: Number of Epochs :::: 17 :::::::: Loss ::::: 0.1200
::

In [135]:
import copy
def validate(model, val_loader):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            y_pred = model(images)
            loss = criterion(y_pred, labels)
            val_loss += loss.item()

            _, predicted = torch.max(y_pred, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return avg_val_loss, accuracy


In [136]:
patience = 5
best_val_accuracy = 0.0
no_improvement_epochs = 0
best_model_weights = None

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0  # To track total loss for the epoch

    for images, labels in train_loader:  # Iterate through training batches
        # Move data to the appropriate device (CPU or GPU)
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)  # Predictions
        loss = criterion(outputs, labels)  # Compute the loss

        # Backward pass
        optimizer.zero_grad()  # Clear gradients from the previous step
        loss.backward()  # Compute gradients
        optimizer.step()  # Update model weights

        # Accumulate the running loss
        running_loss += loss.item()

    # Calculate and print average training loss for this epoch
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    # Validate after each epoch
    avg_val_loss, val_accuracy = validate(model, test_loader)

    # Early stopping logic
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        no_improvement_epochs = 0
        best_model_weights = copy.deepcopy(model.state_dict())
        print("Validation accuracy improved, saving best model...")
    else:
        no_improvement_epochs += 1
        print(f"No improvement in validation accuracy for {no_improvement_epochs} epochs.")

    if no_improvement_epochs >= patience:
        print("Early stopping triggered.")
        break

# Load the best model weights
if best_model_weights is not None:
    model.load_state_dict(best_model_weights)
    print(f"Best model with accuracy {best_val_accuracy:.2f}% loaded.")

Epoch [1/50], Training Loss: 0.1134
Validation Loss: 0.0797, Accuracy: 97.25%
Validation accuracy improved, saving best model...
Epoch [2/50], Training Loss: 0.1122
Validation Loss: 0.0877, Accuracy: 96.98%
No improvement in validation accuracy for 1 epochs.
Epoch [3/50], Training Loss: 0.1127
Validation Loss: 0.0727, Accuracy: 98.14%
Validation accuracy improved, saving best model...
Epoch [4/50], Training Loss: 0.1022
Validation Loss: 0.0830, Accuracy: 96.98%
No improvement in validation accuracy for 1 epochs.
Epoch [5/50], Training Loss: 0.1226
Validation Loss: 0.0922, Accuracy: 96.63%
No improvement in validation accuracy for 2 epochs.
Epoch [6/50], Training Loss: 0.1121
Validation Loss: 0.0888, Accuracy: 96.89%
No improvement in validation accuracy for 3 epochs.
Epoch [7/50], Training Loss: 0.1162
Validation Loss: 0.0771, Accuracy: 97.52%
No improvement in validation accuracy for 4 epochs.
Epoch [8/50], Training Loss: 0.1261
Validation Loss: 0.0927, Accuracy: 96.72%
No improvement

In [137]:
torch.save(model.state_dict(), "satellite_data.pth")

In [145]:
model.load_state_dict(torch.load("satellite_data.pth"))
model.eval()


C:\Users\aarav\AppData\Local\Temp\ipykernel_23260\60183586.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("satellite_data.pth"))


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [119]:
from torchsummary import summary
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [150]:
import torch
from torchvision import models, transforms
from PIL import Image

# Load the model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 4)
model.load_state_dict(torch.load("satellite_data.pth"))
model.eval()

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

######################################################################################################

image_path = r"download.jpg"
image = Image.open(image_path).convert("RGB")
transform = transforms.Compose([
transforms.Grayscale(num_output_channels=3),
transforms.Resize((224,224)),
transforms.RandomHorizontalFlip(),
transforms.RandomRotation(degrees=10),
transforms.RandomResizedCrop(224),
transforms.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2,hue=0.2),
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = transform(image).unsqueeze(0).to(device)

# Make a prediction
with torch.no_grad():
    output = model(input_tensor)

# Interpret the prediction
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()
print(predicted_class)
class_names = ['cloudy','desert','green_area', 'water']
print(f"Predicted: {class_names[predicted_class]} with confidence {probabilities[predicted_class].item():.4f}")


c:\Users\aarav\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aarav\anaconda3\envs\pytorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\aarav\AppData\Local\Temp\ipykernel_23260\3223929553.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorc

0
Predicted: cloudy with confidence 0.6864
